# Reset

In [ ]:
%reset -f

# Import

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import datetime
from PIL import Image

# Image loading and preprocessing

In [ ]:
# convert images to PyTorch tensors
transform = transforms.Compose([
    transforms.Resize((128, 128)), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Values of ImageNet  
])

In [ ]:
# Example image
img = Image.open('/exchange/dspro01/group3/data/train/bobcat/2015_Unit164_Ivan026_img0171.jpg')
img_small = img.resize((512, 512))
img_small

In [ ]:
img_transformed = transform(img)

def denormalize(tensor):
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

to_pil = transforms.ToPILImage()

denormalized_image = to_pil(denormalize(img_transformed.clone()))


In [ ]:
denormalized_image

In [ ]:
# For reproducibility of Transformation
seed = 42
torch.manual_seed(seed)

In [ ]:
data_dir = '/exchange/dspro01/group3/data/train' 
dataset = datasets.ImageFolder(root=data_dir, transform=transform) # assigns labels automatically according to folder's name
# Normalization independent of dataset so applied at root

In [ ]:
num_classes = 12  # number of categories
num_epochs =  25
learning_rate = 0.001 
batch_size = 32

In [ ]:
total_size = len(dataset)
train_size = int(0.8 * total_size)   
val_size = int(0.2 * total_size)     


## Split data

In [ ]:
generator = torch.Generator().manual_seed(seed) # For reproducibility
train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=generator)

In [ ]:
len(train_dataset) == (2400 * 12 * 0.8) # Safety check

## Load data

In [ ]:
# Divides into batches, shuffles data, load in parallel
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=8)

In [ ]:
# Create dataloaders dictionary
dataloaders = {'train': train_loader, 'val': val_loader}


In [ ]:
# Create dataset_sizes dictionary for calculating losses and accuracies
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

# Model loading and transfer learning

In [ ]:
from torchvision import models
import torch.nn as nn

## Load model

In [ ]:
model = models.resnet50(pretrained=True)

num_features = model.fc.in_features # Retrieves the number of input features from the current final layer (fc) of the model
model.fc = nn.Linear(num_features, num_classes) # Replaces the existing final layer with a new nn.Linear layer. Full connected - could add more layers

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss() # Loss function suiting classification problems

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  
# Initial learning rate of 0.001
# weight_decay corresponds to the L2 regularization. Prevents overfitting by penalizing large weight values.

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
# optim = type of learning rate being used
# optimizer = optimizer whose learning rates are being adjusted
# Step size = number of epochs between parameters adjustments
# gamma = The factor by which the learning rate is reduced at each step.

In [ ]:
# Move the model to a GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Training & Validation loop

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, recall_score, precision_score
import numpy as np
import torch

# Initialize metrics storage
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []
val_labels = []
val_predictions = []

# Initialize variables to track the best model
current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
best_val_accuracy = 0.0
best_model_path = f'best_model_{current_time}.pth'


# Track classification metrics
val_f1_scores = []
val_recall = []
val_precision = []
val_conf_matrices = []

# Early Stopping Parameters
patience = 5  # Number of epochs to wait before stopping
early_stopping_counter = 0  # Counter to track how long since the last improvement
best_epoch = 0  # To track the epoch with the best performance

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)
    
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train() # Set model to training mode
        else:
            model.eval()  # Set model to evaluation mode

        running_loss = 0.0
        running_corrects = 0 # number of correct per epoch
        all_preds = []
        all_labels = []

        # Iterate over batches
        for inputs, labels in dataloaders[phase]:
            # Move data to the device GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            # Resets the gradients of all parameters to zero before the backward pass.
            optimizer.zero_grad()

            # Forward pass    
            with torch.set_grad_enabled(phase == 'train'):# if train phase, set gradient
                outputs = model(inputs) # Passes the input in the Resnet model
                # outputs contains the raw predictions of the model for each input
                
                _, preds = torch.max(outputs, 1)
                # Returns the maximum from the outputs, with dimension 1 and a slot for each class
                # Returns 2 tensors: maximum value and its index
                # _ discards the maximum value 
                # Only the index is kept in preds
                
                loss = criterion(outputs, labels)
                # Computes loss function between outputs and labels
                # Refers to the CrossEntropy loss function defined above

                #  Backpropagation and update the model parameters during training.
                if phase == 'train':
                    loss.backward()
                    optimizer.step() # in which direction to go - -gradient

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            all_preds.append(preds.detach().cpu().numpy())
            all_labels.append(labels.detach().cpu().numpy())

        # Convert lists to numpy arrays for metric calculations
        all_preds = np.concatenate(all_preds)
        all_labels = np.concatenate(all_labels)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Store metrics for later plotting
        if phase == 'train':
            train_losses.append(epoch_loss)
            train_accuracies.append(epoch_acc.item())
        else:
            val_predictions.append(all_preds)
            val_labels.append(all_labels)
            val_losses.append(epoch_loss)
            val_accuracies.append(epoch_acc.item())

            # Calculate F1-Score, and confusion matrix for validation phase
            f1 = f1_score(all_labels, all_preds, average='weighted') # from sklearn
            recall = recall_score(all_labels, all_preds, average="weighted")
            precision = precision_score(all_labels, all_preds, average="weighted")
            
            val_f1_scores.append(f1)
            val_recall.append(recall)
            val_precision.append(precision)

            # Confusion Matrix
            conf_matrix = confusion_matrix(all_labels, all_preds)
            val_conf_matrices.append(conf_matrix)

            # Save the best model based on validation accuracy
            if epoch_acc > best_val_accuracy:
                best_val_accuracy = epoch_acc
                best_epoch = epoch + 1
                early_stopping_counter = 0  # Reset the counter
                torch.save({
                    'epoch': epoch + 1,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'accuracy': best_val_accuracy,
                }, best_model_path)
                torch.save(model.state_dict(), f'only_{best_model_path}')
                print(f'Saved best model at epoch {epoch + 1} with validation accuracy: {best_val_accuracy:.4f}')
            else:
                early_stopping_counter += 1  # Increment the counter

    scheduler.step()

    # Check early stopping condition
    if early_stopping_counter >= patience:
        print(f'Early stopping triggered after {epoch + 1} epochs.')
        print(f'Best validation accuracy achieved at epoch {best_epoch}: {best_val_accuracy:.4f}')
        break

print('Training complete')


### Save values

In [ ]:
import pandas as pd

# Create a dictionary for data
data = {
    "epoch": range(1, len(train_losses) + 1),  # Epochs start from 1
    "train_loss": train_losses,
    "val_loss": val_losses,
    "train_accuracy": train_accuracies,
    "val_accuracy": val_accuracies,
    "val_f1_score": val_f1_scores,
    "val_recall": val_recall,
    "val_precision": val_precision,
    "val_conf_matrix": val_conf_matrices,
    "val_labels": val_labels, #
    "val_predictions": val_predictions #
}

# Convert to a DataFrame
df = pd.DataFrame(data)

# Save to a CSV file
file_name = f"training_results_{current_time}.csv"
df.to_csv(file_name, index=False)


# Visualization of training results

In [ ]:
len(train_losses)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

stopped_epoch = best_epoch + patience

# Data from training output
epochs = range(1, stopped_epoch+1)
           
# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
fig.suptitle('Training and Validation Metrics', fontsize=16)

# Plot training & validation loss values
ax1.plot(epochs, train_losses, 'b-', label='Training Loss')
ax1.plot(epochs, val_losses, 'r-', label='Validation Loss')
ax1.set_title('Model Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_ylim(0, 3)
ax1.set_xlim(1)
ax1.legend()
ax1.grid()


# Plot training & validation accuracy values
ax2.plot(epochs, train_accuracies, 'b-', label='Training Accuracy')
ax2.plot(epochs, val_accuracies, 'r-', label='Validation Accuracy')
ax2.set_title('Model Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_ylim(0, 1)
ax2.set_xlim(1)
ax2.set_xticks(ticks=np.arange(1, stopped_epoch + 1, 1))
ax2.legend()
ax2.grid()

# Adjust layout and display the plot
fig.tight_layout()
ax1.plot()
ax2.plot()
fig.show()

# Calculate and print best validation accuracy
best_val_accuracy = max(val_accuracies)
best_epoch = val_accuracies.index(best_val_accuracy) + 1
print(f"Best validation accuracy: {best_val_accuracy:.4f} (Epoch {best_epoch})")

# Calculate and print final training-validation gap
final_train_acc = train_accuracies[-1]
final_val_acc = val_accuracies[-1]
acc_gap = final_train_acc - final_val_acc
print(f"Final accuracy gap (train - val): {acc_gap:.4f}")


In [ ]:
# Plot F1-Score
f, ax = plt.subplots(figsize=(10, 5), sharex=True)
ax.plot(val_f1_scores, label='Validation F1-Score', marker='o', color='blue')
ax.set_xlabel('Epochs', loc='center')
ax.set_ylabel('F1-Score')
ax.set_title('Validation F1-Score Over Epochs')
ax.set_ylim(0, 1)
ax.set_xticks(ticks=np.arange(1, stopped_epoch + 1, 1))
ax.legend()
ax.grid()
f.show()

In [ ]:
import seaborn as sns
print(best_epoch)
# Confusion Matrix for the Best Epoch
conf_matrix = val_conf_matrices[best_epoch - 1]  # Index corresponds to the best epoch
class_names = sorted(['American Black Bear', 'California Ground Squirrel', 'Elk', 'Gray Fox', 'Red Deer', 'Unidentified Deer', 'Bobcat', 'Coyote', 'Empty', 'Mule Deer', 'Striped Skunk', 'Wild Boar'])

f, ax = plt.subplots(figsize=(8, 5), sharex=True)
sns.heatmap(conf_matrix/ (len(val_dataset)/num_classes), annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
ax.set_xlabel('Predicted Labels')
ax.set_ylabel('True Labels')
ax.set_title(f'Confusion Matrix (Epoch {best_epoch})')
f.show()


# Testing

Test set contains 600 pictures per category

In [ ]:
test_dataset = datasets.ImageFolder(root='/exchange/dspro01/group3/data/test', transform=transform) # same transform as train and validation phases
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Load subset of test images for debugging

In [ ]:
from torch.utils.data import Subset

subset_indices = list(range(50))
test_subset = Subset(test_dataset, subset_indices)

# Create a DataLoader for the subset
test_loader_subset = DataLoader(test_subset, batch_size=batch_size, shuffle=False)


## Load trained model and Resnet model

Modify the model to train

In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn
best_model_only = 'only_best_model_2025-01-13_09-42-00.pth'#f'only_{best_model_path}' #'only_best_model_2025-01-04_11-05-04.pth' #  
# Load the pre-trained ResNet model
model = models.resnet50(pretrained=False)

# Modify the 'fc' layer to have 12 output classes
model.fc = nn.Linear(model.fc.in_features, 12)

model.load_state_dict(torch.load(best_model_only, map_location=torch.device('cpu')))
model.eval()


In [ ]:
class_names = sorted(['American Black Bear', 'California Ground Squirrel', 
               'Elk', 'Gray Fox', 'Red Deer', 'Unidentified Deer', 
               'Bobcat', 'Coyote', 'Empty', 'Mule Deer', 
               'Striped Skunk', 'Wild Boar'])
class_mapping = {i: class_names[i] for i in range(12)}

## Model on one picture of the test set

In [ ]:
image_path = '/exchange/dspro01/group3/data/test/bobcat/2015_Unit097_Ivan076_img0742.jpg'
#image_path = '/exchange/dspro01/group3/data/test/gray fox/CA-01_08_13_2015_CA-01_0001873.jpg' #2nd example
image = Image.open(image_path).convert('RGB') 
input_tensor = transform(image).unsqueeze(0) # Add batch dimension

In [ ]:
# Perform inference
with torch.no_grad():  # Disable gradient computation for inference
    output = model(input_tensor)  # Forward pass
    predicted_class = torch.argmax(output, dim=1).item()

print(f"Predicted class: {predicted_class}")
print(f"Predicted class name: {class_mapping[predicted_class]}")


## Model on all test set

In [ ]:
from tqdm import tqdm

all_labels = []
all_predictions = []

correct_predictions = 0
total_samples = 0

with torch.no_grad():  # Disable gradient computation
    for inputs, labels in tqdm(test_loader, desc="Processing Batches", unit="batch"):
        print(f'Loop start')
        outputs = model(inputs)  
        predicted = torch.argmax(outputs, dim=1)
        all_labels.extend(labels.numpy())
        all_predictions.extend(predicted.numpy())  
        
        # Count correct predictions
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)
        
        print('Label and prediction added to the lists')
    print('Testing done.')

# Compute accuracy
accuracy = correct_predictions / total_samples
print(f'Total Accuracy: {accuracy:.2%}')


## Save test data as csv

In [ ]:
# Create a DataFrame
import pandas as pd
df = pd.DataFrame({
    'Label': all_labels,
    'Prediction': all_predictions
})

# Save the DataFrame to a CSV file
filename = f'predictions_model_{best_model_only[-23:-4]}.csv'
df.to_csv(filename, index=False)

## Plot data

In [ ]:
import numpy as np
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)
accuracy = np.mean(all_labels == all_predictions)
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")

#Plot precision for some images
for i in range(3):
    print(f"True label: {class_mapping[all_labels[i*600]]}, Predicted: {class_mapping[all_predictions[i*600]]}")


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
# Safety check
print(len(all_labels) == len(all_predictions))
# Confusion Matrix for the Test set
conf_matrix = confusion_matrix(all_labels, all_predictions)  
class_names = sorted(['American Black Bear', 'California Ground Squirrel', 'Elk', 'Gray Fox', 'Red Deer', 'Unidentified Deer', 'Bobcat', 'Coyote', 'Empty', 'Mule Deer', 'Striped Skunk', 'Wild Boar'])

f, ax = plt.subplots(figsize=(8, 5), sharex=True)
sns.heatmap(conf_matrix/ (len(all_labels)/num_classes), annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
ax.set_xlabel('Predicted Labels')
ax.set_ylabel('True Labels')
ax.set_title(f'Testing - Confusion Matrix (Epoch {best_epoch})')
f.show()